<a href="https://colab.research.google.com/github/MehmetPekerr/reid/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup Project, Environment & Data

# ========================================================= PROJECT SETUP =========================================================
# Clone Repo
!git clone --quiet https://github.com/SurajDonthi/MTMCT-Person-Re-Identification.git
%cd ./MTMCT-Person-Re-Identification/
!pip3 install --quiet -r requirements.txt

# ++++++++++++++++++++++++ D A T A   D O W N L O A D ++++++++++++++++++++++++

# Download Market-1501 Dataset
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ozb7D2cvc-TKVlLVCdYNWmTgZzEvNT4R' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ozb7D2cvc-TKVlLVCdYNWmTgZzEvNT4R" -O ../market\ 1501.zip && rm -rf /tmp/cookies.txt

# Extract Market Dataset
!unzip -qq ../market\ 1501.zip -d ./data/raw/
# Data Cleaning - Fix file names
!find ./data/raw/Market-1501-v15.09.15 -name "*.jpg.jpg" -exec rename -v 's/\.jpg.jpg$/\.jpg/i' {} \;

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# =================================================================================================================================

fatal: destination path 'MTMCT-Person-Re-Identification' already exists and is not an empty directory.
/content/MTMCT-Person-Re-Identification/MTMCT-Person-Re-Identification
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:139: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1ozb7D2cvc-TKVlLVCyNWmTgZzEvNT4R

but Gdown can't. Please check connections and permissions.
replace ./data/raw/Market-1501-v15.09.15/bounding_box_test/-1_c1s1_000401_03.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
from argparse import Namespace

from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
# from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers.test_tube import TestTubeLogger

from mtmct_reid.data import ReIDDataModule
from mtmct_reid.engine import ST_ReID
from mtmct_reid.utils import save_args

import warnings

warnings.filterwarnings('ignore')

# Training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

In [ ]:
args = dict(
    description = "Training and Predicting on Market-1501 Dataset",
    data_dir = "data/raw/Market-1501-v15.09.15",
    dataset = "market",
    log_path = "logs/",

    learning_rate = 0.1,
    criterion = "cross_entropy",
    rerank = False,
    # random_erasing = True,
    # color_jitter = True,

    gpus = 1,
    max_epochs = 60,
    precision = 16,
    train_batchsize = 64,
    val_batchsize = 64,
    test_batchsize = 64,
    limit_val_batches = 1,
    profiler = True,
    progress_bar_refresh_rate = 20
)

args = Namespace(**args)

In [ ]:
tt_logger = TestTubeLogger(save_dir=args.log_path, name="",
                            description=args.description,
                            )

log_dir = Path(tt_logger.save_dir) / f"version_{tt_logger.version}"

checkpoint_dir = log_dir / "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
chkpt_callback = ModelCheckpoint(checkpoint_dir,
                                    monitor='Loss/val_loss/dataloader_idx_2',   # Pytorch-lightning appends extra path
                                    save_last=True,
                                    mode='min',
                                    save_top_k=10,
                                    )

data_module = ReIDDataModule.from_argparse_args(args)

model = ST_ReID(data_module.num_classes, learning_rate=args.learning_rate,
                criterion=args.criterion, rerank=args.rerank)

save_args(args, log_dir)

trainer = Trainer.from_argparse_args(args, logger=[tt_logger],
                                        checkpoint_callback=chkpt_callback,
                                        profiler=True)  # AdvancedProfiler()

#
trainer.fit(model, data_module)

# Testing/Evaluation

Use the model to predict matches from other cameras.

In [ ]:
# Testing the loss & accuracy of the model
trainer.test(model)

# Save the results

In [ ]:
!zip -r /content/experiment_results.zip /content/MTMCT-Person-Re-Identification/logs
from google.colab import files
files.download("/content/experiment_results.zip")